### Text Selection Data Merge

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 45000  # 28 native word end index

# word all usage in twogram
word_use_num_min = 1  # word usage in selected twograms 
word_use_num_max = 2

# n gram select
twogram_select_start = 0
twogram_select_end = 10000

threegram_select_start = 0
threegram_select_end = 10000

fourgram_select_start = 0
fourgram_select_end = 10000

fivegram_select_start = 0
fivegram_select_end = 10000

sixgram_select_start = 0
sixgram_select_end = 10000

sevengram_select_start = 0
sevengram_select_end = 10000

eightgram_select_start = 0
eightgram_select_end = 10000

ninegram_select_start = 0
ninegram_select_end = 10000

tengram_select_start = 0
tengram_select_end = 10000


# prefix suffix file
prefix_suffix = False  # True, False: True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
    
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Text Selection/Result/2-Text Selection Data Merge"

Path(path).mkdir(parents=True, exist_ok=True)

In [ ]:
def strip(df):
    for i in df.columns:
        new_name = i.strip()
        df.rename(columns={f"{i}":f"{new_name}"}, inplace=True)
        df[f"{new_name}"] = df[f"{new_name}"].apply(lambda x: x.strip())
    
    return df

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [ ]:
def word_in_wordgroup(source_word_list, df_target, target_column, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [ ]:
def word_in_wordgroup_simple(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)]   
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Text Data Read

In [ ]:
df_word_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Word_Selected_45000.xlsx")
df_word_select

In [ ]:
#df_twogram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Twogram_Selected_10000.xlsx")
df_twogram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Twogram_Selected.csv")
#df_twogram_select = df_twogram_select.head(1000)
df_twogram_select

In [ ]:
#df_threegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Threegram_Selected_10000.xlsx")
df_threegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Threegram_Selected.csv")
df_threegram_select

In [ ]:
#df_fourgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fourgram_Selected_10000.xlsx")
df_fourgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fourgram_Selected.csv")
df_fourgram_select

In [ ]:
#df_fivegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Fivegram_Selected_10000.xlsx")
df_fivegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Fivegram_Selected.csv")
df_fivegram_select

In [ ]:
#df_sixgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sixgram_Selected_10000.xlsx")
df_sixgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sixgram_Selected.csv")
df_sixgram_select

In [ ]:
#df_sevengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Sevengram_Selected_10000.xlsx")
df_sevengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Sevengram_Selected.csv")
df_sevengram_select

In [ ]:
#df_eightgram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Eightgram_Selected_10000.xlsx")
df_eightgram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Eightgram_Selected.csv")
df_eightgram_select

In [ ]:
#df_ninegram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Ninegram_Selected_10000.xlsx")
df_ninegram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Ninegram_Selected.csv")
df_ninegram_select

In [ ]:
#df_tengram_select = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
#Data/1-Text Selection Data Prepare/Tengram_Selected_10000.xlsx")
df_tengram_select = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Text Selection/\
Data/1-Text Selection Data Prepare/Tengram_Selected.csv")
df_tengram_select

#### Text Data Word-Word Group In Word Group

In [ ]:
word_list = df_word_select["word"].to_list()
twogram_list = df_twogram_select["twogram"].to_list()
threegram_list = df_threegram_select["threegram"].to_list()
fourgram_list = df_fourgram_select["fourgram"].to_list()
fivegram_list = df_fivegram_select["fivegram"].to_list()
sixgram_list = df_sixgram_select["sixgram"].to_list()
sevengram_list = df_sevengram_select["sevengram"].to_list()
eightgram_list = df_eightgram_select["eightgram"].to_list()
ninegram_list = df_ninegram_select["ninegram"].to_list()
tengram_list = df_tengram_select["tengram"].to_list()

##### Word

In [ ]:
# 2 Way Fast
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "twogram"
df_target = df_twogram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

#search_list = list(searchlist)
#result_list = list(resultlist)
#df_word_in_two = pd.DataFrame([search_list,result_list]).transpose()
#df_word_in_two.columns = [f"{search_column_name}",f"{target_column}"]
#df_word_in_two

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_two = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_two

In [ ]:
df_word_in_two[df_word_in_two["word"]=="bu"].sample(20)

In [ ]:
df_word_in_two["word"].nunique()

In [ ]:
df_word_in_two_list = df_word_in_two.groupby(["word"])["twogram"].apply(list).reset_index(name="twogram")
df_word_in_two_list

In [ ]:
#df_word_in_two_list.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_three = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_three

In [ ]:
df_word_in_three_list = df_word_in_three.groupby(["word"])["threegram"].apply(list).reset_index(name="threegram")
df_word_in_three_list

In [ ]:
df_word_in_three["word"].nunique()

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_four = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_four

In [ ]:
df_word_in_four_list = df_word_in_four.groupby(["word"])["fourgram"].apply(list).reset_index(name="fourgram")
df_word_in_four_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_five

In [ ]:
df_word_in_five_list = df_word_in_five.groupby(["word"])["fivegram"].apply(list).reset_index(name="fivegram")
df_word_in_five_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_six

In [ ]:
df_word_in_six_list = df_word_in_six.groupby(["word"])["sixgram"].apply(list).reset_index(name="sixgram")
df_word_in_six_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_seven

In [ ]:
df_word_in_seven_list = df_word_in_seven.groupby(["word"])["sevengram"].apply(list).reset_index(name="sevengram")
df_word_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_eight

In [ ]:
df_word_in_eight_list = df_word_in_eight.groupby(["word"])["eightgram"].apply(list).reset_index(name="eightgram")
df_word_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_nine

In [ ]:
df_word_in_nine_list = df_word_in_nine.groupby(["word"])["ninegram"].apply(list).reset_index(name="ninegram")
df_word_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = word_list
search_column_name = "word"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır nprocs
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_word_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_word_in_ten

In [ ]:
df_word_in_ten_list = df_word_in_ten.groupby(["word"])["tengram"].apply(list).reset_index(name="tengram")
df_word_in_ten_list

In [ ]:
dfs = [df_word_select, df_word_in_two_list, df_word_in_three_list, df_word_in_four_list, df_word_in_five_list, df_word_in_six_list, df_word_in_seven_list, df_word_in_eight_list, df_word_in_nine_list, df_word_in_ten_list]

In [ ]:
df_word_merge = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='left'), dfs)
df_word_merge

In [ ]:
#df_word_merge.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx", index=False)
df_word_merge.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_word_text = pd.read_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.xlsx")
df_word_text = pd.read_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Temp.csv")

In [ ]:
df_word_merge2 = df_word_text[~(df_word_text["twogram"].isnull() & df_word_text["threegram"].isnull() & df_word_text["fourgram"].isnull() & df_word_text["fivegram"].isnull() & \
df_word_text["sixgram"].isnull() & df_word_text["sevengram"].isnull() & df_word_text["eightgram"].isnull() & df_word_text["ninegram"].isnull() & df_word_text["tengram"].isnull())]
df_word_merge2

In [ ]:
#df_word_merge2.to_excel(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.xlsx", index=False)
df_word_merge2.to_csv(f"{lang_folder.capitalize()}_Word_Text_Selection_Result.csv", index=False)

##### Twogram

In [ ]:
# 2 Way Fast
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "threegram"
df_target = df_threegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_three = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_three

In [ ]:
df_two_in_three_list = df_two_in_three.groupby(["twogram"])["threegram"].apply(list).reset_index(name="threegram")
df_two_in_three_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_four = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_four

In [ ]:
df_two_in_four_list = df_two_in_four.groupby(["twogram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_two_in_four_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_five

In [ ]:
df_two_in_five_list = df_two_in_five.groupby(["twogram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_two_in_five_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_six

In [ ]:
df_two_in_six_list = df_two_in_six.groupby(["twogram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_two_in_six_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_seven

In [ ]:
df_two_in_seven_list = df_two_in_seven.groupby(["twogram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_two_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_eight

In [ ]:
df_two_in_eight_list = df_two_in_eight.groupby(["twogram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_two_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_nine

In [ ]:
df_two_in_nine_list = df_two_in_nine.groupby(["twogram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_two_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = twogram_list
search_column_name = "twogram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_two_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_two_in_ten

In [ ]:
df_two_in_ten_list = df_two_in_ten.groupby(["twogram"])["tengram"].apply(list).reset_index(name="tengram")
df_two_in_ten_list

In [ ]:
dfs2 = [df_twogram_select, df_two_in_three_list, df_two_in_four_list, df_two_in_five_list, df_two_in_six_list, df_two_in_seven_list, df_two_in_eight_list, df_two_in_nine_list, df_two_in_ten_list]

In [ ]:
df_twogram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['twogram'], how='left'), dfs2)
df_twogram_merge

In [ ]:
#df_twogram_merge.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx", index=False)
df_twogram_merge.to_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_twogram_text = pd.read_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.xlsx")
df_twogram_text = pd.read_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Temp.csv")

In [ ]:
df_twogram_merge2 = df_twogram_text[~(df_twogram_text["threegram"].isnull() & df_twogram_text["fourgram"].isnull() & df_twogram_text["fivegram"].isnull() & df_twogram_text["sixgram"].isnull() & \
df_twogram_text["sevengram"].isnull() & df_twogram_text["eightgram"].isnull() & df_twogram_text["ninegram"].isnull() & df_twogram_text["tengram"].isnull())]
df_twogram_merge2

In [ ]:
#df_twogram_merge2.to_excel(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.xlsx", index=False)
df_twogram_merge2.to_csv(f"{lang_folder.capitalize()}_Twogram_Text_Selection_Result.csv", index=False)

##### Threegram

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fourgram"
df_target = df_fourgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_four = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_four

In [ ]:
df_three_in_four_list = df_three_in_four.groupby(["threegram"])["fourgram"].apply(list).reset_index(name="fourgram")
df_three_in_four_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_five

In [ ]:
df_three_in_five_list = df_three_in_five.groupby(["threegram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_three_in_five_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_six

In [ ]:
df_three_in_six_list = df_three_in_six.groupby(["threegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_three_in_six_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_seven

In [ ]:
df_three_in_seven_list = df_three_in_seven.groupby(["threegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_three_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_eight

In [ ]:
df_three_in_eight_list = df_three_in_eight.groupby(["threegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_three_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_nine

In [ ]:
df_three_in_nine_list = df_three_in_nine.groupby(["threegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_three_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = threegram_list
search_column_name = "threegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_three_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_three_in_ten

In [ ]:
df_three_in_ten_list = df_three_in_ten.groupby(["threegram"])["tengram"].apply(list).reset_index(name="tengram")
df_three_in_ten_list

In [ ]:
dfs3 = [df_threegram_select, df_three_in_four_list, df_three_in_five_list, df_three_in_six_list, df_three_in_seven_list, df_three_in_eight_list, df_three_in_nine_list, df_three_in_ten_list]

In [ ]:
df_threegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['threegram'], how='left'), dfs3)
df_threegram_merge

In [ ]:
#df_threegram_merge.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx", index=False)
df_threegram_merge.to_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_threegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.xlsx")
df_threegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Temp.csv")

In [ ]:
df_threegram_merge2 = df_threegram_text[~(df_threegram_text["fourgram"].isnull() & df_threegram_text["fivegram"].isnull() & df_threegram_text["sixgram"].isnull() & \
df_threegram_text["sevengram"].isnull() & df_threegram_text["eightgram"].isnull() & df_threegram_text["ninegram"].isnull() & df_threegram_text["tengram"].isnull())]
df_threegram_merge2

In [ ]:
#df_threegram_merge2.to_excel(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.xlsx", index=False)
df_threegram_merge2.to_csv(f"{lang_folder.capitalize()}_Threegram_Text_Selection_Result.csv", index=False)

##### Fourgram

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "fivegram"
df_target = df_fivegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_five = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_five

In [ ]:
df_four_in_five_list = df_four_in_five.groupby(["fourgram"])["fivegram"].apply(list).reset_index(name="fivegram")
df_four_in_five_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_six

In [ ]:
df_four_in_six_list = df_four_in_six.groupby(["fourgram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_four_in_six_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_seven

In [ ]:
df_four_in_seven_list = df_four_in_seven.groupby(["fourgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_four_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_eight

In [ ]:
df_four_in_eight_list = df_four_in_eight.groupby(["fourgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_four_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_nine

In [ ]:
df_four_in_nine_list = df_four_in_nine.groupby(["fourgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_four_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = fourgram_list
search_column_name = "fourgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_four_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_four_in_ten

In [ ]:
df_four_in_ten_list = df_four_in_ten.groupby(["fourgram"])["tengram"].apply(list).reset_index(name="tengram")
df_four_in_ten_list

In [ ]:
dfs4 = [df_fourgram_select, df_four_in_five_list, df_four_in_six_list, df_four_in_seven_list, df_four_in_eight_list, df_four_in_nine_list, df_four_in_ten_list]

In [ ]:
df_fourgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fourgram'], how='left'), dfs4)
df_fourgram_merge

In [ ]:
#df_fourgram_merge.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx", index=False)
df_fourgram_merge.to_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_fourgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.xlsx")
df_fourgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Temp.csv")

In [ ]:
df_fourgram_merge2 = df_fourgram_text[~(df_fourgram_text["fivegram"].isnull() & df_fourgram_text["sixgram"].isnull() & df_fourgram_text["sevengram"].isnull() & \
df_fourgram_text["eightgram"].isnull() & df_fourgram_text["ninegram"].isnull() & df_fourgram_text["tengram"].isnull())]
df_fourgram_merge2

In [ ]:
#df_fourgram_merge2.to_excel(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.xlsx", index=False)
df_fourgram_merge2.to_csv(f"{lang_folder.capitalize()}_Fourgram_Text_Selection_Result.csv", index=False)

##### Fivegram

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sixgram"
df_target = df_sixgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_six = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_six

In [ ]:
df_five_in_six_list = df_five_in_six.groupby(["fivegram"])["sixgram"].apply(list).reset_index(name="sixgram")
df_five_in_six_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_seven

In [ ]:
df_five_in_seven_list = df_five_in_seven.groupby(["fivegram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_five_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_eight

In [ ]:
df_five_in_eight_list = df_five_in_eight.groupby(["fivegram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_five_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_nine

In [ ]:
df_five_in_nine_list = df_five_in_nine.groupby(["fivegram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_five_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = fivegram_list
search_column_name = "fivegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_five_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_five_in_ten

In [ ]:
df_five_in_ten_list = df_five_in_ten.groupby(["fivegram"])["tengram"].apply(list).reset_index(name="tengram")
df_five_in_ten_list

In [ ]:
dfs5 = [df_fivegram_select, df_five_in_six_list, df_five_in_seven_list, df_five_in_eight_list, df_five_in_nine_list, df_five_in_ten_list]

In [ ]:
df_fivegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['fivegram'], how='left'), dfs5)
df_fivegram_merge

In [ ]:
#df_fivegram_merge.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx", index=False)
df_fivegram_merge.to_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_fivegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.xlsx")
df_fivegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Temp.csv")

In [ ]:
df_fivegram_merge2 = df_fivegram_text[~(df_fivegram_text["sixgram"].isnull() & df_fivegram_text["sevengram"].isnull() & \
df_fivegram_text["eightgram"].isnull() & df_fivegram_text["ninegram"].isnull() & df_fivegram_text["tengram"].isnull())]
df_fivegram_merge2

In [ ]:
#df_fivegram_merge2.to_excel(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.xlsx", index=False)
df_fivegram_merge2.to_csv(f"{lang_folder.capitalize()}_Fivegram_Text_Selection_Result.csv", index=False)

##### Sixgram

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "sevengram"
df_target = df_sevengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_seven = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_seven

In [ ]:
df_six_in_seven_list = df_six_in_seven.groupby(["sixgram"])["sevengram"].apply(list).reset_index(name="sevengram")
df_six_in_seven_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_eight

In [ ]:
df_six_in_eight_list = df_six_in_eight.groupby(["sixgram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_six_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_nine

In [ ]:
df_six_in_nine_list = df_six_in_nine.groupby(["sixgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_six_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = sixgram_list
search_column_name = "sixgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_six_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_six_in_ten

In [ ]:
df_six_in_ten_list = df_six_in_ten.groupby(["sixgram"])["tengram"].apply(list).reset_index(name="tengram")
df_six_in_ten_list

In [ ]:
dfs6 = [df_sixgram_select, df_six_in_seven_list, df_six_in_eight_list, df_six_in_nine_list, df_six_in_ten_list]

In [ ]:
df_sixgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sixgram'], how='left'), dfs6)
df_sixgram_merge

In [ ]:
#df_sixgram_merge.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx", index=False)
df_sixgram_merge.to_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_sixgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.xlsx")
df_sixgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Temp.csv")

In [ ]:
df_sixgram_merge2 = df_sixgram_text[~(df_sixgram_text["sevengram"].isnull() & df_sixgram_text["eightgram"].isnull() & df_sixgram_text["ninegram"].isnull() & df_sixgram_text["tengram"].isnull())]
df_sixgram_merge2

In [ ]:
#df_sixgram_merge2.to_excel(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.xlsx", index=False)
df_sixgram_merge2.to_csv(f"{lang_folder.capitalize()}_Sixgram_Text_Selection_Result.csv", index=False)

##### Sevengram

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "eightgram"
df_target = df_eightgram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_eight = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_seven_in_eight

In [ ]:
df_seven_in_eight_list = df_seven_in_eight.groupby(["sevengram"])["eightgram"].apply(list).reset_index(name="eightgram")
df_seven_in_eight_list

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_seven_in_nine

In [ ]:
df_seven_in_nine_list = df_seven_in_nine.groupby(["sevengram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_seven_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = sevengram_list
search_column_name = "sevengram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_seven_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_seven_in_ten

In [ ]:
df_seven_in_ten_list = df_seven_in_ten.groupby(["sevengram"])["tengram"].apply(list).reset_index(name="tengram")
df_seven_in_ten_list

In [ ]:
dfs7 = [df_sevengram_select, df_seven_in_eight_list, df_seven_in_nine_list, df_seven_in_ten_list]

In [ ]:
df_sevengram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['sevengram'], how='left'), dfs7)
df_sevengram_merge

In [ ]:
#df_sevengram_merge.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx", index=False)
df_sevengram_merge.to_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_sevengram_text = pd.read_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.xlsx")
df_sevengram_text = pd.read_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Temp.csv")

In [ ]:
df_sevengram_merge2 = df_sevengram_text[~(df_sevengram_text["eightgram"].isnull() & df_sevengram_text["ninegram"].isnull() & df_sevengram_text["tengram"].isnull())]
df_sevengram_merge2

In [ ]:
#df_sevengram_merge2.to_excel(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.xlsx", index=False)
df_sevengram_merge2.to_csv(f"{lang_folder.capitalize()}_Sevengram_Text_Selection_Result.csv", index=False)

##### Eightgram

In [ ]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "ninegram"
df_target = df_ninegram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_nine = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_eight_in_nine

In [ ]:
df_eight_in_nine_list = df_eight_in_nine.groupby(["eightgram"])["ninegram"].apply(list).reset_index(name="ninegram")
df_eight_in_nine_list

In [ ]:
# parameter
source_wordgroup_list = eightgram_list
search_column_name = "eightgram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_eight_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_eight_in_ten

In [ ]:
df_eight_in_ten_list = df_eight_in_ten.groupby(["eightgram"])["tengram"].apply(list).reset_index(name="tengram")
df_eight_in_ten_list

In [ ]:
dfs8 = [df_eightgram_select, df_eight_in_nine_list, df_eight_in_ten_list]

In [ ]:
df_eightgram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['eightgram'], how='left'), dfs8)
df_eightgram_merge

In [ ]:
#df_eightgram_merge.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx", index=False)
df_eightgram_merge.to_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_eightgram_text = pd.read_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.xlsx")
df_eightgram_text = pd.read_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Temp.csv")

In [ ]:
df_eightgram_merge2 = df_eightgram_text[~(df_eightgram_text["ninegram"].isnull() & df_eightgram_text["tengram"].isnull())]
df_eightgram_merge2

In [ ]:
#df_eightgram_merge2.to_excel(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.xlsx", index=False)
df_eightgram_merge2.to_csv(f"{lang_folder.capitalize()}_Eightgram_Text_Selection_Result.csv", index=False)

##### Ninegram

In [ ]:
# parameter
source_wordgroup_list = ninegram_list
search_column_name = "ninegram"
target_column = "tengram"
df_target = df_tengram_select

# function
searchlist = []
resultlist = []
manager = multiprocessing.Manager()
resultlist = manager.list()
searchlist = manager.list()

def word_in_wordgroup(wordgroup):    
    try:
        word_in_word_cluster = df_target[df_target[f"{target_column}"].str.contains(fr"(?:\s|^){wordgroup}(?:\s|$)", na=True)].iloc[0:2000,]
        for result in word_in_word_cluster[f"{target_column}"]:
            searchlist.append(wordgroup)
            resultlist.append(result)
    except:
        pass

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, source_wordgroup_list) # string_word liste

In [ ]:
search_list = list(searchlist)
result_list = list(resultlist)
df_nine_in_ten = pd.DataFrame({f"{search_column_name}":search_list,f"{target_column}":result_list})
df_nine_in_ten

In [ ]:
df_nine_in_ten_list = df_nine_in_ten.groupby(["ninegram"])["tengram"].apply(list).reset_index(name="tengram")
df_nine_in_ten_list

In [ ]:
dfs9 = [df_ninegram_select, df_nine_in_ten_list]

In [ ]:
df_ninegram_merge = reduce(lambda  left,right: pd.merge(left,right, on=['ninegram'], how='left'), dfs9)
df_ninegram_merge

In [ ]:
#df_ninegram_merge.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx", index=False)
df_ninegram_merge.to_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.csv", index=False)

In [ ]:
#df_ninegram_text = pd.read_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.xlsx")
df_ninegram_text = pd.read_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Temp.csv")

In [ ]:
df_ninegram_merge2 = df_ninegram_text[~(df_ninegram_text["tengram"].isnull())]
df_ninegram_merge2

In [ ]:
#df_ninegram_merge2.to_excel(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.xlsx", index=False)
df_ninegram_merge2.to_csv(f"{lang_folder.capitalize()}_Ninegram_Text_Selection_Result.csv", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Text_Selection_Result*")
#output_file = glob.glob(f"*_Selected_*.xlsx")
output_file

In [ ]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass